In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/Portfolio/Project 1/train_clean.csv")
train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,company_type,training_hours,target,experience_clean,last_new_job_clean,company_size_clean,training_hours_clean,training_hours_bin,last_job_change_bin,company_size_bin
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,Unknown,36,1.0,21.0,1.0,300.0,36.0,Moderate (26-50),≤1 year,Mid
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,Pvt Ltd,47,0.0,15.0,5.0,75.0,47.0,Moderate (26-50),4–5 years,Small
2,11561,city_21,0.624,Unknown,No relevent experience,Full time course,Graduate,STEM,Unknown,83,0.0,5.0,0.0,300.0,83.0,High (51-100),≤1 year,Mid
3,33241,city_115,0.789,Unknown,No relevent experience,Unknown,Graduate,Business Degree,Pvt Ltd,52,1.0,0.0,0.0,300.0,52.0,High (51-100),≤1 year,Mid
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,Funded Startup,8,0.0,21.0,4.0,75.0,8.0,Low (0-25),4–5 years,Small


In [ ]:
train.shape

(19158, 18)

In [ ]:
train.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'company_type', 'training_hours', 'target',
       'experience_clean', 'last_new_job_clean', 'company_size_clean',
       'training_hours_clean', 'training_hours_bin', 'last_job_change_bin',
       'company_size_bin'],
      dtype='object')

In [ ]:
train["target"].mean()

np.float64(0.24934753105752167)

In [ ]:
# Basic baseline
n_rows = len(train)
overall_rate_py = train["target"].mean()

print(f"Rows: {n_rows:,}")
print(f"Overall job-change rate (Python): {overall_rate_py:.6f}")
print(train["target"].value_counts(dropna=False))

Rows: 19,158
Overall job-change rate (Python): 0.249348
target
0.0    14381
1.0     4777
Name: count, dtype: int64


In [ ]:
import sqlite3
import pandas as pd

# Create an in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# Load dataframe into SQL table
train.to_sql("train", conn, index=False, if_exists="replace")

# SQL version of overall job-change rate
sql = """
SELECT
COUNT(*) AS n_rows,
AVG(target) AS overall_job_change_rate
FROM train;
"""

result = pd.read_sql_query(sql, conn)
result

,n_rows,overall_job_change_rate
0,19158,0.249348


In [ ]:
print("Python:", train["target"].mean())
print("SQL:", result.loc[0, "overall_job_change_rate"])

Python: 0.24934753105752167
SQL: 0.24934753105752167


In [ ]:
# Python version for job-change rate by career stage
career_stage_py = (
    train
    .groupby("last_job_change_bin")
    .agg(
        n=("target", "size"),
        job_change_rate=("target", "mean")
    )
    .reset_index()
    .sort_values("last_job_change_bin")
)

career_stage_py

,last_job_change_bin,n,job_change_rate
0,2–3 years,3924,0.237258
1,4–5 years,4319,0.191711
2,≤1 year,10915,0.276500


In [ ]:
# SQL version for job-change rate by career stage
sql = """
SELECT
last_job_change_bin,
COUNT(*) AS n,
AVG(target) AS job_change_rate
FROM train
GROUP BY last_job_change_bin
ORDER BY last_job_change_bin;
"""

career_stage_sql = pd.read_sql_query(sql, conn)
career_stage_sql

,last_job_change_bin,n,job_change_rate
0,2–3 years,3924,0.237258
1,4–5 years,4319,0.191711
2,≤1 year,10915,0.276500


In [ ]:
# Parity Check for job-change rate by career stage
parity_check = career_stage_py.merge(
    career_stage_sql,
    on="last_job_change_bin",
    suffixes=("_py", "_sql")
)

parity_check["rate_diff"] = (

    parity_check["job_change_rate_py"] - parity_check["job_change_rate_sql"]
).abs()

parity_check

,last_job_change_bin,n_py,job_change_rate_py,n_sql,job_change_rate_sql,rate_diff
0,2–3 years,3924,0.237258,3924,0.237258,0.0
1,4–5 years,4319,0.191711,4319,0.191711,0.0
2,≤1 year,10915,0.276500,10915,0.276500,0.0


In [ ]:
assert parity_check["rate_diff"].max() < 1e-12
print("Parity confirmed: job-change rate by career stage")

Parity confirmed: job-change rate by career stage


In [ ]:
# Python version for job-change rate by training exposure
training_py = (
    train
    .groupby("training_hours_bin")
    .agg(
        n=("target", "size"),
        job_change_rate=("target", "mean")
    )
    .reset_index()
    .sort_values("job_change_rate", ascending=False)
)

training_py

,training_hours_bin,n,job_change_rate
2,Moderate (26-50),4798,0.258858
1,Low (0-25),5395,0.252456
0,High (51-100),5097,0.245438
3,Very High (101+),3868,0.238366


In [ ]:
# SQL version for job-change rate by training exposure
sql = """
SELECT
training_hours_bin,
COUNT(*) AS n,
AVG(target) AS job_change_rate
FROM train
GROUP BY training_hours_bin
ORDER BY job_change_rate DESC;
"""

training_sql = pd.read_sql_query(sql, conn)
training_sql

,training_hours_bin,n,job_change_rate
0,Moderate (26-50),4798,0.258858
1,Low (0-25),5395,0.252456
2,High (51-100),5097,0.245438
3,Very High (101+),3868,0.238366


In [ ]:
# Parity check for job-change rate by training exposure
training_parity = training_py.merge(
    training_sql,
    on="training_hours_bin",
    suffixes=("_py", "_sql")
)

training_parity["n_diff"] = (
    training_parity["n_py"] - training_parity["n_sql"]
).abs()
training_parity["rate_diff"] = (
    training_parity["job_change_rate_py"] - training_parity["job_change_rate_sql"]
).abs()

training_parity

,training_hours_bin,n_py,job_change_rate_py,n_sql,job_change_rate_sql,n_diff,rate_diff
0,Moderate (26-50),4798,0.258858,4798,0.258858,0,0.0
1,Low (0-25),5395,0.252456,5395,0.252456,0,0.0
2,High (51-100),5097,0.245438,5097,0.245438,0,0.0
3,Very High (101+),3868,0.238366,3868,0.238366,0,0.0


In [ ]:
assert training_parity["n_diff"].max() == 0, "Count mismatch"
assert training_parity["rate_diff"].max() < 1e-12, "Rate mismatch"
print("Parity confirmed: job-change rate by training exposure")

Parity confirmed: job-change rate by training exposure


In [ ]:
# Bin education levels
higher_ed = {"Graduate", "Masters", "Phd"}

train["education_group"] = train["education_level"].where(
    train["education_level"].isin(higher_ed),
    "Other/Unknown"
)

#QA check
train["education_group"].value_counts(dropna=False)

,count
education_group,
Graduate,11598
Masters,4361
Other/Unknown,2785
Phd,414


In [ ]:
"education_group" in train.columns

True

In [ ]:
# Recreate the SQL table so it includes the new column
train.to_sql("train", conn, index=False, if_exists="replace")

19158

In [ ]:
# Python version for job-change rate by education group
education_py = (
    train
    .groupby("education_group")
    .agg(
        n=("target", "size"),
        job_change_rate=("target", "mean")
    )
    .reset_index()
    .sort_values("job_change_rate", ascending=False)
)

education_py

,education_group,n,job_change_rate
0,Graduate,11598,0.279790
1,Masters,4361,0.214400
2,Other/Unknown,2785,0.193537
3,Phd,414,0.140097


In [ ]:
pd.read_sql_query("PRAGMA table_info(train);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,enrollee_id,INTEGER,0,None,0
1,1,city,TEXT,0,None,0
2,2,city_development_index,REAL,0,None,0
3,3,gender,TEXT,0,None,0
4,4,relevent_experience,TEXT,0,None,0
5,5,enrolled_university,TEXT,0,None,0
6,6,education_level,TEXT,0,None,0
7,7,major_discipline,TEXT,0,None,0
8,8,company_type,TEXT,0,None,0
9,9,training_hours,INTEGER,0,None,0


In [ ]:
# SQL version of job-change rate by education group
sql = """
SELECT
education_group,
COUNT(*) AS n,
AVG(target) AS job_change_rate
FROM train
GROUP BY education_group
ORDER BY job_change_rate DESC;
"""

education_sql = pd.read_sql_query(sql, conn)
education_sql

,education_group,n,job_change_rate
0,Graduate,11598,0.279790
1,Masters,4361,0.214400
2,Other/Unknown,2785,0.193537
3,Phd,414,0.140097


In [ ]:
# Parity check of job-change rate by education group
education_parity = education_py.merge(
    education_sql,
    on="education_group",
    suffixes=("_py", "_sql")
)

education_parity["n_diff"] = (
    education_parity["n_py"] - education_parity["n_sql"]
).abs()
education_parity["rate_diff"] = (
    education_parity["job_change_rate_py"] - education_parity["job_change_rate_sql"]
).abs()

education_parity

,education_group,n_py,job_change_rate_py,n_sql,job_change_rate_sql,n_diff,rate_diff
0,Graduate,11598,0.279790,11598,0.279790,0,0.0
1,Masters,4361,0.214400,4361,0.214400,0,0.0
2,Other/Unknown,2785,0.193537,2785,0.193537,0,0.0
3,Phd,414,0.140097,414,0.140097,0,0.0


In [ ]:
assert education_parity["n_diff"].max() == 0, "Count mismatch"
assert education_parity["rate_diff"].max() < 1e-12, "Rate mismatch"
print("Parity confirmed: job-change rate by education group")

Parity confirmed: job-change rate by education group


In [ ]:
# Python version of job-change rate by valdated persona
persona_py = (
    train
    .groupby(["last_job_change_bin", "training_hours_bin"])
    .agg(
        n=("target", "size"),
        job_change_rate=("target", "mean")
    )
    .reset_index()
    .sort_values("job_change_rate", ascending=False)
)

persona_py

,last_job_change_bin,training_hours_bin,n,job_change_rate
10,≤1 year,Moderate (26-50),2741,0.287486
9,≤1 year,Low (0-25),3096,0.282946
8,≤1 year,High (51-100),2852,0.275596
11,≤1 year,Very High (101+),2226,0.255166
2,2–3 years,Moderate (26-50),975,0.249231
3,2–3 years,Very High (101+),786,0.237913
1,2–3 years,Low (0-25),1093,0.232388
0,2–3 years,High (51-100),1070,0.230841
7,4–5 years,Very High (101+),856,0.195093
6,4–5 years,Moderate (26-50),1082,0.195009


In [ ]:
# SQL version of job-change by validated persona
sql = """
SELECT
last_job_change_bin,
training_hours_bin,
COUNT(*) AS n,
AVG(target) AS job_change_rate
FROM train
GROUP BY last_job_change_bin, training_hours
ORDER BY job_change_rate DESC;
"""

persona_sql = pd.read_sql_query(sql, conn)
persona_sql

,last_job_change_bin,training_hours_bin,n,job_change_rate
0,2–3 years,Very High (101+),1,1.0
1,2–3 years,Very High (101+),2,1.0
2,2–3 years,Very High (101+),1,1.0
3,2–3 years,Very High (101+),2,1.0
4,2–3 years,Very High (101+),1,1.0
...,...,...,...,...
706,≤1 year,Very High (101+),3,0.0
707,≤1 year,Very High (101+),5,0.0
708,≤1 year,Very High (101+),9,0.0
709,≤1 year,Very High (101+),4,0.0


In [ ]:
# Parity check for job-change rate by validated persona
persona_parity = persona_py.merge(
    persona_sql,
    on=["last_job_change_bin", "training_hours_bin"],
    suffixes=("_py", "_sql")
)

persona_parity["n_diff"] = (
    persona_parity["n_py"] - persona_parity["n_sql"]).abs()
persona_parity["rate_diff"] = (
    persona_parity["job_change_rate_py"] - persona_parity["job_change_rate_sql"]
).abs()

persona_parity

,last_job_change_bin,training_hours_bin,n_py,job_change_rate_py,n_sql,job_change_rate_sql,n_diff,rate_diff
0,≤1 year,Moderate (26-50),2741,0.287486,107,0.392523,2634,0.105037
1,≤1 year,Moderate (26-50),2741,0.287486,167,0.359281,2574,0.071795
2,≤1 year,Moderate (26-50),2741,0.287486,133,0.345865,2608,0.058378
3,≤1 year,Moderate (26-50),2741,0.287486,119,0.327731,2622,0.040245
4,≤1 year,Moderate (26-50),2741,0.287486,127,0.322835,2614,0.035348
...,...,...,...,...,...,...,...,...
706,4–5 years,High (51-100),1175,0.185532,32,0.093750,1143,0.091782
707,4–5 years,High (51-100),1175,0.185532,32,0.062500,1143,0.123032
708,4–5 years,High (51-100),1175,0.185532,20,0.050000,1155,0.135532
709,4–5 years,High (51-100),1175,0.185532,10,0.000000,1165,0.185532


In [ ]:
assert persona_parity["n_diff"].max() == 0, "Count mismatch"
assert persona_parity["rate_diff"].max() < 1e-12, "Rate mismatch"
print("Parity confirmed: job-change rate by validated persona")

AssertionError: Count mismatch

In [ ]:
# Show only rows with mismatched counts
persona_parity.loc[persona_parity["n_diff"] != 0].sort_values("n_diff", ascending=False).head(20)

,last_job_change_bin,training_hours_bin,n_py,job_change_rate_py,n_sql,job_change_rate_sql,n_diff,rate_diff
37,≤1 year,Low (0-25),3096,0.282946,7,0.285714,3089,0.002769
41,≤1 year,Low (0-25),3096,0.282946,62,0.274194,3034,0.008752
38,≤1 year,Low (0-25),3096,0.282946,63,0.285714,3033,0.002769
40,≤1 year,Low (0-25),3096,0.282946,75,0.280000,3021,0.002946
31,≤1 year,Low (0-25),3096,0.282946,90,0.300000,3006,0.017054
25,≤1 year,Low (0-25),3096,0.282946,106,0.358491,2990,0.075545
36,≤1 year,Low (0-25),3096,0.282946,108,0.287037,2988,0.004091
27,≤1 year,Low (0-25),3096,0.282946,121,0.338843,2975,0.055897
46,≤1 year,Low (0-25),3096,0.282946,121,0.247934,2975,0.035012
29,≤1 year,Low (0-25),3096,0.282946,122,0.311475,2974,0.028530


In [ ]:
for col in ["last_job_change_bin", "training_hours_bin"]:
  train[col] = (
      train[col]
      .astype(str)    # force string
      .str.strip()    # remove hidden whitespace
      .str.replace("-", "-", regex=False)   # normalize dashes if present
  )

In [ ]:
train.to_sql("train", conn, index=False, if_exists="replace")

19158

In [ ]:
# Re-run SQL version of job-change rate by validated personas
sql = """
SELECT
last_job_change_bin,
training_hours_bin,
COUNT(*) AS n,
AVG(target) AS job_change_rate
FROM train
GROUP BY last_job_change_bin, training_hours_bin
ORDER BY job_change_rate DESC;
"""

persona_sql = pd.read_sql_query(sql, conn)
persona_sql


,last_job_change_bin,training_hours_bin,n,job_change_rate
0,≤1 year,Moderate (26-50),2741,0.287486
1,≤1 year,Low (0-25),3096,0.282946
2,≤1 year,High (51-100),2852,0.275596
3,≤1 year,Very High (101+),2226,0.255166
4,2–3 years,Moderate (26-50),975,0.249231
5,2–3 years,Very High (101+),786,0.237913
6,2–3 years,Low (0-25),1093,0.232388
7,2–3 years,High (51-100),1070,0.230841
8,4–5 years,Very High (101+),856,0.195093
9,4–5 years,Moderate (26-50),1082,0.195009


In [ ]:
# Re-run parity check for job-change rate by validated personas
persona_parity = persona_py.merge(
    persona_sql,
    on=["last_job_change_bin", "training_hours_bin"],
    suffixes=("_py", "_sql")
)

persona_parity["n_diff"] = persona_parity["n_py"] - persona_parity["n_sql"].abs()
persona_parity["rate_diff"] = (
    persona_parity["job_change_rate_py"] - persona_parity["job_change_rate_sql"]
).abs()

persona_parity

,last_job_change_bin,training_hours_bin,n_py,job_change_rate_py,n_sql,job_change_rate_sql,n_diff,rate_diff
0,≤1 year,Moderate (26-50),2741,0.287486,2741,0.287486,0,0.0
1,≤1 year,Low (0-25),3096,0.282946,3096,0.282946,0,0.0
2,≤1 year,High (51-100),2852,0.275596,2852,0.275596,0,0.0
3,≤1 year,Very High (101+),2226,0.255166,2226,0.255166,0,0.0
4,2–3 years,Moderate (26-50),975,0.249231,975,0.249231,0,0.0
5,2–3 years,Very High (101+),786,0.237913,786,0.237913,0,0.0
6,2–3 years,Low (0-25),1093,0.232388,1093,0.232388,0,0.0
7,2–3 years,High (51-100),1070,0.230841,1070,0.230841,0,0.0
8,4–5 years,Very High (101+),856,0.195093,856,0.195093,0,0.0
9,4–5 years,Moderate (26-50),1082,0.195009,1082,0.195009,0,0.0


In [ ]:
assert persona_parity["n_diff"].max() == 0, "Count mismatch"
assert persona_parity["rate_diff"].max() < 1e-12, "Rate mismatch"
print("Parity confirmed: job-change rate by validated persona")

Parity confirmed: job-change rate by validated persona


In [ ]:
# Freeze personas and create reference table
persona_baseline = (
    train
    .groupby(["last_job_change_bin", "training_hours_bin"])
    .agg(
        n=("target", "size"),
        job_change_rate=("target", "mean")
    )
    .reset_index()
    .sort_values("job_change_rate", ascending=False)
)

persona_baseline

,last_job_change_bin,training_hours_bin,n,job_change_rate
10,≤1 year,Moderate (26-50),2741,0.287486
9,≤1 year,Low (0-25),3096,0.282946
8,≤1 year,High (51-100),2852,0.275596
11,≤1 year,Very High (101+),2226,0.255166
2,2–3 years,Moderate (26-50),975,0.249231
3,2–3 years,Very High (101+),786,0.237913
1,2–3 years,Low (0-25),1093,0.232388
0,2–3 years,High (51-100),1070,0.230841
7,4–5 years,Very High (101+),856,0.195093
6,4–5 years,Moderate (26-50),1082,0.195009


In [ ]:
# Create training intensity
train["training_intensity"] = (
    train["training_hours_clean"] / train["experience_clean"]
    )

# Guardrail: remove invalid/infinite values
train.loc[train["experience_clean"] <= 0, "training_intensity"] = None

# Quick QA
train["training_intensity"].describe()

,training_intensity
count,18636.000000
mean,11.669155
std,19.574691
min,0.047619
25%,2.363636
50%,5.500000
75%,13.000000
max,328.000000


In [ ]:
# Create training intensity bins
train["training_intensity_bin"] = pd.cut(
    train["training_intensity"],
    bins=[0, 3, 8, 15, float("inf")],
    labels=[
        "Low (< 3 hours/year)",
        "Medium (3-8 hours/year)",
        "High (8-15 hours/year)",
        "Very High (> 15 hours/year)"
    ]
)

#QA
train["training_intensity_bin"].value_counts(dropna=False)

,count
training_intensity_bin,
Low (< 3 hours/year),5962
Medium (3-8 hours/year),5696
Very High (> 15 hours/year),3972
High (8-15 hours/year),3006
NaN,522


In [ ]:
train.groupby("training_intensity_bin")["target"].agg(
    n="count",
    job_change_rate="mean"
).reset_index()

/tmp/ipython-input-3897028570.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train.groupby("training_intensity_bin")["target"].agg(


,training_intensity_bin,n,job_change_rate
0,Low (< 3 hours/year),5962,0.200436
1,Medium (3-8 hours/year),5696,0.228055
2,High (8-15 hours/year),3006,0.275449
3,Very High (> 15 hours/year),3972,0.306647


In [ ]:
%whos

Variable           Type          Data/Info
------------------------------------------
assign_persona     function      <function assign_persona at 0x7e8025275120>
career_stage_py    DataFrame       last_job_change_bin    <...>r  10915         0.276500
career_stage_sql   DataFrame       last_job_change_bin    <...>r  10915         0.276500
col                str           training_hours_bin
conn               Connection    <sqlite3.Connection object at 0x7e7fe686e200>
drive              module        <module 'google.colab.dri<...>s/google/colab/drive.py'>
education_parity   DataFrame       education_group   n_py <...> \n3       0        0.0  
education_py       DataFrame       education_group      n <...>d    414         0.140097
education_sql      DataFrame       education_group      n <...>d    414         0.140097
files              module        <module 'google.colab.fil<...>s/google/colab/files.py'>
higher_ed          set           {'Masters', 'Phd', 'Graduate'}
map_career_stage   

In [ ]:
import sqlite3
import pandas as pd

# Create an in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# Load existing dataframe (train) into SQL
train.to_sql(
    "train_clean",
    conn,
    index=False,
    if_exists="replace"
)

19158

In [ ]:
# Persona SQL

sql_query = """
WITH base AS (
  SELECT
  enrollee_id,
  target,
  education_level,
  training_hours_clean,
  training_hours_bin,
  last_job_change_bin,

  -- # Education amplifier flag
  CASE
    WHEN education_level IN ("Graduate", "Masters", "Phd") THEN 1
    ELSE 0
  END AS higher_education_flag
FROM train_clean
),
personas AS (
  SELECT
  *,
  CASE
  WHEN last_job_change_bin = '≤1 year'
  THEN 'Early-Career Explorer'
  WHEN last_job_change_bin = '2-3 years'
  THEN 'Mid-Career Plateau Risk'
  WHEN last_job_change_bin IN ('4-5 years', '>5 years', '5+ years')
  THEN 'Anchored Contributor'
  ELSE 'Not Classified'
  END AS mobility_persona
  FROM base
)
SELECT *
FROM personas;
"""

persona_sql = pd.read_sql_query(sql_query, conn)
persona_sql.head()

,enrollee_id,target,education_level,training_hours_clean,training_hours_bin,last_job_change_bin,higher_education_flag,mobility_persona
0,8949,1.0,Graduate,36.0,Moderate (26-50),≤1 year,1,Early-Career Explorer
1,29725,0.0,Graduate,47.0,Moderate (26-50),4–5 years,1,Not Classified
2,11561,0.0,Graduate,83.0,High (51-100),≤1 year,1,Early-Career Explorer
3,33241,1.0,Graduate,52.0,High (51-100),≤1 year,1,Early-Career Explorer
4,666,0.0,Masters,8.0,Low (0-25),4–5 years,1,Not Classified


In [ ]:
# Distribution check
persona_sql['mobility_persona'].value_counts(dropna=False)

# Decision relevance check
persona_sql.groupby('mobility_persona')['target'].mean().sort_values(ascending=False)

,target
mobility_persona,
Early-Career Explorer,0.276500
Not Classified,0.213393


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

base = "/content/drive/MyDrive/Portfolio/Project 1"
print("Exists:", os.path.exists(base))
print("Contents:", os.listdir(base) if os.path.exists(base) else "Folder not found")

Exists: True
Contents: ['train_clean.csv']


In [ ]:
import os

matches = []
for root, dirs, files in os.walk("/content/drive/MyDrive"):
  if "train_clean.csv" in files:
    matches.append(os.path.join(root, "train_clean.csv"))

matches[:10], len(matches)

(['/content/drive/MyDrive/Portfolio/Project 1/train_clean.csv'], 1)

In [ ]:
import pandas as pd

# Load cleaned dataset from Drive
CSV_PATH = "/content/drive/MyDrive/Portfolio/Project 1/train_clean.csv"
train = pd.read_csv(CSV_PATH)
train.head()

# Sanity checks
train.shape
train.columns.tolist()

['enrollee_id',
 'city',
 'city_development_index',
 'gender',
 'relevent_experience',
 'enrolled_university',
 'education_level',
 'major_discipline',
 'company_type',
 'training_hours',
 'target',
 'experience_clean',
 'last_new_job_clean',
 'company_size_clean',
 'training_hours_clean',
 'training_hours_bin',
 'last_job_change_bin',
 'company_size_bin']

In [ ]:
# Inspect exact values, including hidden characters
raw_bins = train["last_job_change_bin"].dropna().astype(str).unique().tolist()

for v in sorted(raw_bins):
  print(repr(v))

'2–3 years'
'4–5 years'
'≤1 year'


In [ ]:
# Create a normalized working column (no function)
train['last_job_change_bin_norm'] = (
    train['last_job_change_bin']
    .astype(str)
    .str.strip()
    .str.replace('–', '-', regex=False)   # EN dash > hyphen
    .str.replace('≤', '<=', regex=False)  # normalize symbol
)

In [ ]:
# Map to canonical categories
train['last_job_change_bin_norm'] = train['last_job_change_bin_norm'].map({
    '<=1 year': '<=1 year',
    '2-3 years': '2-3 years',
    '4-5 years': '4-5 years',
}).fillna('Other')

In [ ]:
# Sanity check
train['last_job_change_bin_norm'].value_counts(dropna=False)

,count
last_job_change_bin_norm,
<=1 year,10915
4-5 years,4319
2-3 years,3924


In [ ]:
import sqlite3

conn = sqlite3.connect("memory:")

# Load normalized dataframe into SQL
train.to_sql(
    "train_clean",
    conn,
    index=False,
    if_exists="replace"
)

19158

In [ ]:
# SQL Persona
sql_query = """
WITH base AS (
  SELECT
  enrollee_id,
  target,
  education_level,
  training_hours_clean,
  training_hours_bin,
  last_job_change_bin_norm,

  CASE
  WHEN education_level IN ("Graduate", "Masters", "Phd") THEN 1
  ELSE 0
  END AS higher_education_flag
FROM train_clean
),
personas AS (
  SELECT
  *,
  CASE
  WHEN last_job_change_bin_norm = '<=1 year'
  THEN 'Early-Career Explorer'
  WHEN last_job_change_bin_norm = '2-3 years'
  THEN 'Mid-Career Plateau Risk'
  WHEN last_job_change_bin_norm = '4-5 years'
  THEN 'Anchored Contributor'
  ELSE 'Not Classified'
  END AS mobility_persona
  FROM base
)
SELECT *
FROM personas;
"""

persona_sql = pd.read_sql_query(sql_query, conn)

In [ ]:
# Coverage check
persona_sql['mobility_persona'].value_counts(normalize=True).round(3)

# Decision relevance check
persona_sql.groupby('mobility_persona')['target'].mean().sort_values(ascending=False)

,target
mobility_persona,
Early-Career Explorer,0.276500
Mid-Career Plateau Risk,0.237258
Anchored Contributor,0.191711


In [ ]:
# Persona size (count + %)
persona_sql['mobility_persona'].value_counts().to_frame('n').assign(
    pct=lambda x: (x['n'] / x['n'].sum()).round(3)
)

,n,pct
mobility_persona,,
Early-Career Explorer,10915,0.570
Anchored Contributor,4319,0.225
Mid-Career Plateau Risk,3924,0.205


In [ ]:
# Overlap check (should be exactly one persona per enrollee_id)
overlap_check = (
    persona_sql.groupby('enrollee_id')['mobility_persona']
    .nunique()
    .value_counts()
)

overlap_check

,count
mobility_persona,
1,19158


In [ ]:
# Alternative cut: collapse tenure into 2 groups
alt_persona = persona_sql.copy()

alt_persona['alt_mobility_persona'] = (
    alt_persona['last_job_change_bin_norm'].map({
        '<=1 year': 'Recent Movers (<=1 year)',
        '2-3 years': 'Later Movers (>=2 years)',
        '4-5 years': 'Later Movers (>=2 years)'
    }).fillna('Other')
)

In [ ]:
# Alternative cut: size check
alt_persona['alt_mobility_persona'].value_counts(normalize=True).round(3)

,proportion
alt_mobility_persona,
Recent Movers (<=1 year),0.57
Later Movers (>=2 years),0.43


In [ ]:
# Alternative cut: Decision relevance check
alt_persona.groupby('alt_mobility_persona')['target'].mean().sort_values(ascending=False)

,target
alt_mobility_persona,
Recent Movers (<=1 year),0.276500
Later Movers (>=2 years),0.213393


In [ ]:
# Build persona-ready dataset for Tableau / SQL
tableau_cols = [
    'enrollee_id',
    'target',
    'mobility_persona',
    'last_job_change_bin_norm',
    'training_hours_bin',
    'education_level',
    'experience_clean',
    'company_size_bin'
]

tableau_df = (
    train
    .merge(
        persona_sql[['enrollee_id', 'mobility_persona']],
        on='enrollee_id',
        how='left'
    )
    [tableau_cols]
)

In [ ]:
# Row count check
len(tableau_df), len(train)

(19158, 19158)

In [ ]:
# Persona completeness
tableau_df['mobility_persona'].isna().sum()

np.int64(0)

In [ ]:
# Outcome distibution by persona
tableau_df.groupby('mobility_persona')['target'].mean().round(3)

,target
mobility_persona,
Anchored Contributor,0.192
Early-Career Explorer,0.277
Mid-Career Plateau Risk,0.237


In [ ]:
# Export Tableau-ready CSV
OUT_PATH = "/content/drive/MyDrive/Portfolio/Project 1/train_personas_tableau_ready.csv"
tableau_df.to_csv(OUT_PATH, index=False)

OUT_PATH

'/content/drive/MyDrive/Portfolio/Project 1/train_personas_tableau_ready.csv'